<a href="https://colab.research.google.com/github/VCGI/notebooks/blob/main/vt_contour_clipper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VT Contour-Clipper
This tool is designed to help you easily select an Area of Interest (AOI) anywhere in Vermont and export 1-ft contours for the selected area.

Press the play button to run the first cell which will load a map for you to identify the area. Optionally, you can also choose to include parcel data and aerial imagery for the area. You'll be given the option to download the processed data in various formats (like GeoJSON, Shapefile, GeoPackage, DXF, or GeoParquet). Follow the steps below to download the data.

In [ ]:
#@title Select AOI, Load Contours & Clip (with optional Parcels & Imagery)


print("Setting things up. This may take a moment.")
import ipywidgets as widgets # Import ipywidgets first for the progress bar
from IPython.display import display, HTML # Import display for the progress bar and HTML for custom styling
import urllib.parse # For URL encoding

# --- Setup Progress Bar ---
setup_progress = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Starting Setup:',
    bar_style='info',
    style={'bar_color': '#17A2B8'}, # Blue color for info
    orientation='horizontal',
    layout=widgets.Layout(width='auto')
)
display(setup_progress)

# --- Stage 1: Install libraries ---
setup_progress.value = 10
setup_progress.description = 'Loading'
# Install libraries (run this cell once)
# rasterio is needed for COG access and clipping
!pip install geopandas leafmap pyarrow shapely mapclassify s3fs rasterio ezdxf --quiet &> /dev/null
setup_progress.value = 50
setup_progress.description = 'Importing libs...'

# --- Stage 2: Import libraries ---
import geopandas
import leafmap.leafmap as leafmap
import pandas as pd
from shapely.geometry import Polygon, Point, box
import tempfile
import shutil
import os
import json
import time
import ezdxf
from ezdxf.enums import TextEntityAlignment
import pyarrow
from ipyleaflet import DrawControl, GeoData
import rasterio
from rasterio.mask import mask as rio_mask # Specific import for mask function
import numpy as np # For array manipulation if needed

setup_progress.value = 100
setup_progress.bar_style = 'success'
setup_progress.description = 'Library Setup Complete!'
print("Finished Library Setup")
print("-" * 50)


print("Loading up a map for you to select an area.")
# --- Configuration ---
TOWN_BOUNDARIES_URL = "https://s3.us-east-2.amazonaws.com/vtopendata-prd/Boundaries/vt-town-topo.json"
BASE_CONTOUR_S3_PATH = "s3://vtopendata-prd/Elevation/Contours/"
PARCEL_DATA_URL_BASE = "https://services1.arcgis.com/BkFxaEFNwHqX3tAw/arcgis/rest/services/FS_VCGI_OPENDATA_Cadastral_VTPARCELS_poly_standardized_parcels_SP_v1/FeatureServer/0"
IMAGERY_COG_URL = "https://s3.us-east-2.amazonaws.com/vtopendata-prd/Imagery/STATEWIDE_2024_30cm_LeafOFF_4Band.tif"


VERMONT_CENTER = [44.0, -72.699997]
INITIAL_ZOOM = 7

# --- Global variables to store data and AOI ---
gdf_towns = None
gdf_contours = None
gdf_parcels_raw = None
gdf_parcels_clipped = None
clipped_imagery_path = None # Path to the GeoTIFF in os.getcwd()
aoi_gdf = None
clipped_gdf = None
selected_town_names_str = None

# --- Output widgets (defined here for global access, displayed in relevant cells) ---
town_load_status_label = widgets.Label(value="Town Boundaries: Not loaded")
town_load_progress = widgets.IntProgress(
    value=0, min=0, max=100, description='Loading Towns:', bar_style='',
    style={'bar_color': '#4CAF50'}, orientation='horizontal',
    layout=widgets.Layout(width='auto', visibility='hidden')
)
aoi_status_label = widgets.Label(value="AOI Status: Not defined")
contour_load_status_label = widgets.Label(value="Contour Data: Not loaded")
parcel_load_status_label = widgets.Label(value="Parcel Data: Not included or not loaded")
imagery_status_label = widgets.Label(value="Imagery: Not included or not processed")
clip_status_label = widgets.Label(value="Clipping Status: Not performed")

# --- UI Elements ---
button_layout = widgets.Layout(width='auto', min_width='200px')
include_parcels_toggle = widgets.Checkbox(
    value=False, description='Include Parcel Data?', disabled=False, indent=False
)
include_imagery_toggle = widgets.Checkbox(
    value=False, description='Include 2024 30cm Leaf-Off Imagery?', disabled=False, indent=False # Updated description
)

# Widgets for Download Cell (now Cell 2)
output_format_dropdown = widgets.Dropdown(
    options=['GeoJSON', 'Shapefile', 'GeoPackage', 'DXF', 'GeoParquet'],
    value='GeoJSON', description='Output Format:', disabled=False,
)
download_button = widgets.Button(
    description="Download Data", disabled=True, layout=button_layout
)
download_output_area = widgets.Output()

print("Configuration and global widgets initialized.")
print("Finished Configuration & Global Setup")
print("-" * 50)


print("Hang in there while town boundaries are loaded.")
# --- Function to load Vermont town boundaries ---
def load_town_boundaries_data(url):
    global gdf_towns, town_load_progress
    town_load_progress.value = 0
    town_load_progress.layout.visibility = 'visible'
    town_load_progress.description = 'Loading Towns:'
    try:
        gdf_towns_temp = geopandas.read_file(url)
        if gdf_towns_temp.crs is None:
            gdf_towns_temp = gdf_towns_temp.set_crs("EPSG:4326", allow_override=True)
        elif gdf_towns_temp.crs.to_string() != "EPSG:4326":
            gdf_towns_temp = gdf_towns_temp.to_crs("EPSG:4326")
        gdf_towns = gdf_towns_temp
        if 'TOWN' not in gdf_towns.columns:
            potential_fields = [col for col in gdf_towns.columns if 'town' in col.lower() or 'name' in col.lower()]
            if potential_fields:
                 print(f"Warning: 'TOWN' field not found. Using '{potential_fields[0]}' as town name field. Please verify.")
                 gdf_towns.rename(columns={potential_fields[0]: 'TOWN'}, inplace=True)
            else:
                print("Error: 'TOWN' field (or similar) not found in town boundaries data.")
                gdf_towns = None; return
        town_load_progress.description = 'Towns Loaded!'
        town_load_progress.bar_style = 'success'
    except Exception as e:
        print(f"Error loading town boundaries: {e}"); gdf_towns = None
        town_load_progress.description = 'Error!'; town_load_progress.bar_style = 'danger'

display(town_load_progress)
load_town_boundaries_data(TOWN_BOUNDARIES_URL)

# --- Map Creation and AOI Logic ---
if 'leafmap' not in globals(): print("ERROR: Essential libraries not found.")
else:
    m = leafmap.Map(center=VERMONT_CENTER, zoom=INITIAL_ZOOM, search_control=False, draw_control=False, measure_control=False, fullscreen_control=True, attribution_control=True)
    m.add_basemap("HYBRID")
    town_boundaries_layer_name = "Vermont Town Boundaries"
    if gdf_towns is not None and not gdf_towns.empty:
        town_style = {'color': '#cccccc', 'weight': 1, 'fillColor': '#cccccc', 'fillOpacity': 0.0, 'opacity': 0.65}
        m.add_gdf(gdf_towns, layer_name=town_boundaries_layer_name, style=town_style, hover_style=town_style.copy(), info_mode=None, zoom_to_layer=False)
        for layer in m.layers:
            if hasattr(layer, 'name') and layer.name == town_boundaries_layer_name and isinstance(layer, GeoData):
                layer.hover_style = layer.style.copy(); break
    else: print("Town boundaries not loaded or empty. Cannot add to map.")

    custom_draw_control = DrawControl(marker={}, circlemarker={}, circle={}, polyline={}, polygon={'shapeOptions': {'fillColor': '#3388ff','color': '#3388ff','fillOpacity': 0.2},'allowIntersection': False}, rectangle={'shapeOptions': {'fillColor': '#3388ff','color': '#3388ff','fillOpacity': 0.2}}, edit=True, remove=True)
    m.add_control(custom_draw_control)
    m.draw_control = custom_draw_control

    # --- Function to load parcel data ---
    def load_parcel_data(aoi_boundary_gdf, base_url):
        global gdf_parcels_raw, parcel_load_status_label
        parcel_load_status_label.value = "Parcel Data: Loading... this may take a moment."
        gdf_parcels_raw = None
        try:
            aoi_wgs84 = aoi_boundary_gdf.to_crs(epsg=4326) if aoi_boundary_gdf.crs.to_epsg() != 4326 else aoi_boundary_gdf
            bounds = aoi_wgs84.total_bounds
            params = {'where': '1=1', 'outFields': '*', 'geometry': f'{bounds[0]},{bounds[1]},{bounds[2]},{bounds[3]}', 'geometryType': 'esriGeometryEnvelope', 'inSR': '4326', 'spatialRel': 'esriSpatialRelIntersects', 'outSR': '4326', 'f': 'geojson'}
            query_url = f"{base_url}/query?{urllib.parse.urlencode(params)}"
            print(f"Querying parcel data with URL: {query_url}")
            gdf_parcels_temp = geopandas.read_file(query_url)
            if gdf_parcels_temp.empty:
                parcel_load_status_label.value = "Parcel Data: No parcels found in AOI's bbox."; return None
            if gdf_parcels_temp.crs is None: gdf_parcels_temp = gdf_parcels_temp.set_crs("EPSG:4326", allow_override=True)
            elif gdf_parcels_temp.crs.to_string().upper() != "EPSG:4326": gdf_parcels_temp = gdf_parcels_temp.to_crs("EPSG:4326")
            gdf_parcels_raw = gdf_parcels_temp
            parcel_load_status_label.value = f"Parcel Data: Loaded {len(gdf_parcels_raw)} raw parcels."
            return gdf_parcels_raw
        except Exception as e:
            parcel_load_status_label.value = f"Parcel Data: Error loading - {e}"; print(f"Error loading parcel data: {e}"); traceback.print_exc(); return None

    # --- Function to clip imagery to AOI ---
    def clip_imagery_to_aoi(aoi_geom_gdf, cog_url):
        global clipped_imagery_path, imagery_status_label
        imagery_status_label.value = "Imagery: Clipping (3-band RGB)... this may take a moment."

        if clipped_imagery_path and os.path.exists(clipped_imagery_path):
            try: os.remove(clipped_imagery_path); print(f"Removed previous temp imagery: {clipped_imagery_path}")
            except Exception as e_remove: print(f"Could not remove previous temp imagery '{clipped_imagery_path}': {e_remove}")
        clipped_imagery_path = None

        try:
            aoi_geom_gdf_proj = aoi_geom_gdf.to_crs(epsg=4326) if aoi_geom_gdf.crs.to_epsg() != 4326 else aoi_geom_gdf
            geoms = [feature["geometry"] for feature in json.loads(aoi_geom_gdf_proj.to_json())['features']]

            with rasterio.open(cog_url) as src:
                if src.count < 3:
                    imagery_status_label.value = f"Imagery: Source COG has only {src.count} bands. Cannot create 3-band RGB."
                    print(f"Error: Source COG has only {src.count} bands. Cannot create 3-band RGB.")
                    return None

                if src.crs and src.crs.to_string().upper() != aoi_geom_gdf_proj.crs.to_string().upper():
                    print(f"Reprojecting AOI from {aoi_geom_gdf_proj.crs} to COG CRS {src.crs}")
                    aoi_for_mask = aoi_geom_gdf_proj.to_crs(src.crs)
                    geoms_for_mask = [feature["geometry"] for feature in json.loads(aoi_for_mask.to_json())['features']]
                else:
                    geoms_for_mask = geoms

                # Mask the data (will read all bands present in src by default)
                out_image_all_bands, out_transform = rio_mask(src, geoms_for_mask, crop=True, all_touched=True, filled=True, nodata=src.nodata if src.nodata is not None else 0)
                out_meta = src.meta.copy()

                # Select first 3 bands for RGB output
                if out_image_all_bands.shape[0] >= 3:
                    out_image_rgb = out_image_all_bands[:3, :, :]
                else: # Should have been caught by src.count check, but as a safeguard
                    imagery_status_label.value = f"Imagery: Masked image has less than 3 bands ({out_image_all_bands.shape[0]})."
                    print(f"Error: Masked image has less than 3 bands ({out_image_all_bands.shape[0]}).")
                    return None


            out_meta.update({
                "driver": "GTiff",
                "height": out_image_rgb.shape[1],
                "width": out_image_rgb.shape[2],
                "transform": out_transform,
                "crs": src.crs,
                "count": 3 # Explicitly set band count to 3 for RGB
            })
            if 'nodata' in out_meta and not isinstance(out_meta['nodata'], (int, float)):
                del out_meta['nodata']


            timestamp = int(time.time())
            temp_imagery_filename = f"clipped_imagery_rgb_aoi_{timestamp}.tif"
            output_path_in_cwd = os.path.join(os.getcwd(), temp_imagery_filename)

            with rasterio.open(output_path_in_cwd, "w", **out_meta) as dest:
                dest.write(out_image_rgb) # Write only the 3 selected bands

            clipped_imagery_path = output_path_in_cwd
            imagery_status_label.value = f"Imagery: Clipped (3-band RGB) to {temp_imagery_filename}"
            print(f"Imagery (3-band RGB) clipped and saved to: {clipped_imagery_path}")
            return clipped_imagery_path
        except Exception as e:
            imagery_status_label.value = f"Imagery: Error clipping - {e}"
            print(f"Error clipping imagery: {e}")
            import traceback
            traceback.print_exc()
            clipped_imagery_path = None
            return None


    # --- Function to load town-specific contour data for merging ---
    def load_town_contour_data_for_merge(town_name):
        if not town_name: print(f"Contour Data: Town name not provided for {town_name}."); return None, False
        contour_filename = f"{town_name.upper().replace(' ', '_')}_ElevationContours_CN1TGEN_SP.parquet"
        contour_url = f"{BASE_CONTOUR_S3_PATH}{contour_filename}"
        try:
            gdf_temp = geopandas.read_parquet(contour_url, storage_options={'anon': True})
            gdf_temp.dropna(subset=['geometry'], inplace=True)
            if 'geometry' not in gdf_temp.columns or gdf_temp.empty:
                if gdf_temp.empty: print(f"Note: No valid geometries for {town_name} after None removal."); return None, False
                print(f"No geometry column for {town_name}."); return None, False
            if gdf_temp.crs is None: gdf_temp = gdf_temp.set_crs("EPSG:4326", allow_override=True)
            elif gdf_temp.crs.to_string() != "EPSG:4326": gdf_temp = gdf_temp.to_crs("EPSG:4326")
            return gdf_temp, True
        except Exception as e:
            if 'No such file' not in str(e) and 'Is a directory' not in str(e) and '404' not in str(e): print(f"Error loading contours for {town_name}: {e}")
            else: print(f"Note: No contour file for {town_name} (expected for some).")
            return None, False

    # --- Function to clip data (called after contours are loaded) ---
    def clip_data_to_aoi_internal():
        global gdf_contours, gdf_parcels_raw, aoi_gdf, clipped_gdf, gdf_parcels_clipped, clipped_imagery_path
        global clip_status_label, download_button, parcel_load_status_label, imagery_status_label

        download_output_area.clear_output()
        clip_status_label.value = "Clipping Status: Processing..."
        download_button.disabled = True
        contours_clipped_successfully = False
        parcels_clipped_successfully = False
        imagery_processed_successfully = False

        # Clip Contours
        if gdf_contours is not None and not gdf_contours.empty and aoi_gdf is not None:
            try:
                gdf_contours_transformed = gdf_contours.to_crs(epsg=4326) if gdf_contours.crs.to_epsg() != 4326 else gdf_contours.copy()
                gdf_contours_transformed.dropna(subset=['geometry'], inplace=True)
                if not gdf_contours_transformed.is_valid.all():
                    print("..."); gdf_contours_transformed['geometry'] = gdf_contours_transformed.geometry.buffer(0)
                if not aoi_gdf.is_valid.all(): print("Warning: Invalid AOI geometry. Buffering by 0."); aoi_gdf['geometry'] = aoi_gdf.geometry.buffer(0)

                if not gdf_contours_transformed.empty:
                    clipped_gdf_temp = geopandas.clip(gdf_contours_transformed, aoi_gdf)
                    if not clipped_gdf_temp.empty:
                        clipped_gdf = clipped_gdf_temp; contours_clipped_successfully = True
                        clip_status_label.value = f"Clipping: Contours ({len(clipped_gdf)} features) clipped."
                    else: clip_status_label.value = "Clipping: No contours in AOI."; clipped_gdf = None
                else: print("Error: No valid contours to clip."); clipped_gdf = None
            except Exception as e: clip_status_label.value = f"Clipping Error (Contours): {e}"; print(f"Contour clip error: {e}"); traceback.print_exc(); clipped_gdf = None
        elif aoi_gdf is None: clip_status_label.value = "Clipping: AOI not defined."
        else: clip_status_label.value = "Clipping: Contours not loaded/empty."

        # Clip Parcels
        if include_parcels_toggle.value and gdf_parcels_raw is not None and not gdf_parcels_raw.empty and aoi_gdf is not None:
            parcel_load_status_label.value = "Parcel Data: Clipping..."
            try:
                gdf_parcels_transformed = gdf_parcels_raw.to_crs(epsg=4326) if gdf_parcels_raw.crs.to_epsg() != 4326 else gdf_parcels_raw.copy()
                gdf_parcels_transformed.dropna(subset=['geometry'], inplace=True)
                if not gdf_parcels_transformed.is_valid.all(): print("..."); gdf_parcels_transformed['geometry'] = gdf_parcels_transformed.geometry.buffer(0)
                if not gdf_parcels_transformed.empty:
                    gdf_parcels_clipped_temp = geopandas.clip(gdf_parcels_transformed, aoi_gdf)
                    if not gdf_parcels_clipped_temp.empty:
                        gdf_parcels_clipped = gdf_parcels_clipped_temp; parcels_clipped_successfully = True
                        parcel_load_status_label.value = f"Parcels: Clipped ({len(gdf_parcels_clipped)} features)."
                    else: parcel_load_status_label.value = "Parcels: No parcels in AOI."; gdf_parcels_clipped = None
                else: print("No valid parcels to clip."); gdf_parcels_clipped = None
            except Exception as e: parcel_load_status_label.value = f"Parcels Clip Error: {e}"; print(f"Parcel clip error: {e}"); traceback.print_exc(); gdf_parcels_clipped = None
        elif include_parcels_toggle.value : parcel_load_status_label.value = "Parcels: Not loaded or empty, skipping clip."

        # Clip Imagery
        if include_imagery_toggle.value and aoi_gdf is not None:
            clip_imagery_to_aoi(aoi_gdf, IMAGERY_COG_URL)
            if clipped_imagery_path and os.path.exists(clipped_imagery_path):
                imagery_processed_successfully = True
            else:
                imagery_status_label.value = "Imagery: Clipping failed or no output."
        elif include_imagery_toggle.value: imagery_status_label.value = "Imagery: AOI not defined, skipping clip."


        # Update final status and download button
        final_statuses = []
        if contours_clipped_successfully: final_statuses.append("Contours processed.")
        elif clipped_gdf is None and gdf_contours is not None : final_statuses.append("No contours in AOI.")

        if include_parcels_toggle.value:
            if parcels_clipped_successfully: final_statuses.append("Parcels processed.")
            elif gdf_parcels_clipped is None and gdf_parcels_raw is not None: final_statuses.append("No parcels in AOI.")
            elif gdf_parcels_raw is None : final_statuses.append("Parcels not loaded.")

        if include_imagery_toggle.value:
            if imagery_processed_successfully: final_statuses.append("Imagery processed.")
            else: final_statuses.append("Imagery processing failed or no output.")

        if final_statuses:
            clip_status_label.value = "Clipping Complete: " + " ".join(final_statuses)
        else:
            clip_status_label.value = "Clipping Status: No data processed or AOI not defined."

        if contours_clipped_successfully or parcels_clipped_successfully or imagery_processed_successfully:
            download_button.disabled = False
        else:
            download_button.disabled = True


    # --- Function to handle drawn AOI, load contours, and clip ---
    def handle_aoi_load_and_clip(geo_json_feature):
        global aoi_gdf, gdf_towns, selected_town_names_str, gdf_contours, gdf_parcels_raw, gdf_parcels_clipped, clipped_imagery_path
        global contour_load_status_label, clip_status_label, parcel_load_status_label, imagery_status_label, download_button

        aoi_status_label.value = "AOI Status: Processing drawn feature..."
        # Reset all data
        gdf_contours = None; clipped_gdf = None
        gdf_parcels_raw = None; gdf_parcels_clipped = None
        # Clean up imagery from os.getcwd() if it exists from a previous run
        if clipped_imagery_path and os.path.exists(clipped_imagery_path):
            try: os.remove(clipped_imagery_path); print(f"Cleaned up previous imagery: {clipped_imagery_path}")
            except Exception as e_remove: print(f"Could not remove old imagery '{clipped_imagery_path}': {e_remove}")
        clipped_imagery_path = None # Reset path
        selected_town_names_str = None

        contour_load_status_label.value = "Contour Data: Not loaded"
        parcel_load_status_label.value = "Parcel Data: Not included" if not include_parcels_toggle.value else "Parcel Data: Pending..."
        imagery_status_label.value = "Imagery: Not included" if not include_imagery_toggle.value else "Imagery: Pending..."
        clip_status_label.value = "Clipping Status: Not performed"
        download_button.disabled = True

        try:
            if geo_json_feature['geometry']['type'] == 'Polygon':
                coords = geo_json_feature['geometry']['coordinates']
                polygon_geom = Polygon(coords[0])
                aoi_gdf_temp = geopandas.GeoDataFrame([{'id':1, 'geometry': polygon_geom}], crs="EPSG:4326")
                aoi_gdf = aoi_gdf_temp
                aoi_status_label.value = f"AOI Status: AOI defined. Identifying intersecting town(s)..."

                if gdf_towns is None: aoi_status_label.value = "AOI: Town boundaries not loaded."; return
                if 'TOWN' not in gdf_towns.columns: aoi_status_label.value = "AOI: 'TOWN' field missing."; return

                intersecting_towns_gdf = gdf_towns[gdf_towns.intersects(aoi_gdf.geometry.iloc[0])]
                # Contour Loading
                if not intersecting_towns_gdf.empty:
                    town_names_to_load = intersecting_towns_gdf['TOWN'].unique().tolist()
                    aoi_status_label.value = f"AOI: Intersects town(s): {', '.join(town_names_to_load)}. Loading contours...Thank you for your patience."
                    all_town_contours_list = []
                    successfully_loaded_town_names = []
                    failed_to_load_town_names = []
                    contour_load_progress_multi = None
                    if town_names_to_load:
                        contour_load_progress_multi = widgets.IntProgress(
                            value=0, min=0, max=len(town_names_to_load), description='Loading Contours:',
                            bar_style='info', style={'bar_color': '#007bff'}, layout=widgets.Layout(width='auto')
                        )
                        display(contour_load_progress_multi)
                    for i, town_name in enumerate(town_names_to_load):
                        temp_gdf, success = load_town_contour_data_for_merge(town_name)
                        if success and temp_gdf is not None and not temp_gdf.empty:
                            all_town_contours_list.append(temp_gdf)
                            successfully_loaded_town_names.append(town_name)
                        elif not success:
                            failed_to_load_town_names.append(town_name)
                        if contour_load_progress_multi: contour_load_progress_multi.value = i + 1
                    if contour_load_progress_multi:
                        contour_load_progress_multi.bar_style = 'success'
                        contour_load_progress_multi.description = 'Contours Loaded!'

                    if all_town_contours_list:
                        gdf_contours = pd.concat(all_town_contours_list, ignore_index=True)
                        if gdf_contours.crs is None and all_town_contours_list and all_town_contours_list[0].crs is not None:
                             gdf_contours = gdf_contours.set_crs(all_town_contours_list[0].crs)
                        elif gdf_contours.crs is None:
                             gdf_contours = gdf_contours.set_crs("EPSG:4326", allow_override=True)
                        selected_town_names_str = ", ".join(successfully_loaded_town_names)
                        load_msg = f"Contours for {selected_town_names_str}: Loaded ({len(gdf_contours)} lines)."
                        if failed_to_load_town_names: load_msg += f" (Failed/No data for: {', '.join(failed_to_load_town_names)})."
                        contour_load_status_label.value = load_msg
                    else:
                        contour_load_status_label.value = "Contours: No data loaded for identified town(s)."
                else: contour_load_status_label.value = "Contours: No town identified for AOI."

                # Parcel Loading
                if include_parcels_toggle.value:
                    load_parcel_data(aoi_gdf, PARCEL_DATA_URL_BASE)
                else: gdf_parcels_raw = None; gdf_parcels_clipped = None

                # Imagery Clipping will be handled within clip_data_to_aoi_internal
                if include_imagery_toggle.value:
                    imagery_status_label.value = "Imagery: Will be processed during clipping."
                else:
                    clipped_imagery_path = None

                # Call clipping which now handles imagery internally as well
                clip_data_to_aoi_internal()

            else: aoi_status_label.value = "AOI: Please draw a polygon/rectangle."
        except Exception as e:
            aoi_status_label.value = f"AOI Error: {e}"; print(f"Error in handle_aoi: {e}"); traceback.print_exc()

    # --- Buttons for AOI interaction ---
    confirm_aoi_load_clip_button = widgets.Button(description="Confirm AOI & Process", layout=button_layout)
    clear_aoi_button = widgets.Button(description="Clear AOI & Data", layout=button_layout)

    def on_confirm_aoi_load_clip_button_clicked(b):
        if m.draw_control.data:
            last_drawn_feature = m.draw_control.data[-1]
            if last_drawn_feature and last_drawn_feature.get('geometry'): handle_aoi_load_and_clip(last_drawn_feature)
            else: aoi_status_label.value = "AOI: Last drawn feature invalid."
        else: aoi_status_label.value = "AOI: No AOI drawn."

    def on_clear_aoi_button_clicked(b):
        global aoi_gdf, gdf_contours, gdf_parcels_raw, gdf_parcels_clipped, selected_town_names_str, clipped_gdf, clipped_imagery_path
        m.draw_control.clear()
        aoi_gdf = None; gdf_contours = None; clipped_gdf = None
        gdf_parcels_raw = None; gdf_parcels_clipped = None
        # Clean up imagery from os.getcwd() if it exists
        if clipped_imagery_path and os.path.exists(clipped_imagery_path):
            try: os.remove(clipped_imagery_path); print(f"Removed temp imagery: {clipped_imagery_path}")
            except Exception as e_remove: print(f"Could not remove temp imagery '{clipped_imagery_path}': {e_remove}")
        clipped_imagery_path = None
        selected_town_names_str = None
        aoi_status_label.value = "AOI: Not defined. Data cleared."
        contour_load_status_label.value = "Contours: Not loaded"
        parcel_load_status_label.value = "Parcels: Not included/loaded"
        imagery_status_label.value = "Imagery: Not included/processed"
        clip_status_label.value = "Clipping: Not performed"
        download_button.disabled = True
        print("Cleared drawn AOIs and all loaded/clipped data.")

    if hasattr(confirm_aoi_load_clip_button, '_click_handlers'): confirm_aoi_load_clip_button._click_handlers.callbacks = []
    if hasattr(clear_aoi_button, '_click_handlers'): clear_aoi_button._click_handlers.callbacks = []
    confirm_aoi_load_clip_button.on_click(on_confirm_aoi_load_clip_button_clicked)
    clear_aoi_button.on_click(on_clear_aoi_button_clicked)

    print("Instructions for AOI selection and processing:")
    print("1. Pan/zoom to find your area. Town boundaries are shown.")
    print("2. Use drawing tools (polygon/rectangle) for your AOI.")
    print("3. Optionally, check 'Include Parcel Data?' and/or 'Include Imagery (3-band RGB)?'.")
    print("4. Click 'Confirm AOI & Process'.")
    print("5. To redraw, click 'Clear AOI & Data' first.")

    display(m)
    display(widgets.HBox([include_parcels_toggle, include_imagery_toggle]))
    display(widgets.HBox([confirm_aoi_load_clip_button, clear_aoi_button]))
    display(aoi_status_label); display(contour_load_status_label); display(parcel_load_status_label); display(imagery_status_label); display(clip_status_label)

print("-" * 50)
print("Cell 1 (Setup, AOI Selection, and Processing) is complete. If successful, you can proceed to Cell 2 to download.")



In [ ]:
#@title Select Format and Download Clipped Data

# Ensure previous cell (Cell 1) has been run and global variables are available.
if 'clipped_gdf' not in globals() or \
   'gdf_parcels_clipped' not in globals() or \
   'clipped_imagery_path' not in globals() or \
   'download_output_area' not in globals() or \
   'output_format_dropdown' not in globals() or \
   'download_button' not in globals() or \
   'widgets' not in globals() or \
   'include_parcels_toggle' not in globals() or \
   'include_imagery_toggle' not in globals():
    print("ERROR: Essential variables for downloading not found.")
    print("Please ensure Cell 1 (Setup, AOI Selection, and Processing) has been run successfully.")
else:
    def on_download_button_clicked(b):
        global clipped_gdf, gdf_parcels_clipped, clipped_imagery_path, download_output_area, selected_town_names_str, include_parcels_toggle, include_imagery_toggle

        download_output_area.clear_output()

        has_contours = clipped_gdf is not None and not clipped_gdf.empty
        parcels_requested_and_clipped = include_parcels_toggle.value and gdf_parcels_clipped is not None and not gdf_parcels_clipped.empty
        imagery_requested_and_clipped = include_imagery_toggle.value and clipped_imagery_path is not None and os.path.exists(clipped_imagery_path)

        if not has_contours and not parcels_requested_and_clipped and not imagery_requested_and_clipped:
            with download_output_area:
                print("No clipped contour, parcel, or imagery data available to download. Please process AOI in Cell 1.")
            return

        selected_format = output_format_dropdown.value
        messages_to_display = [f"Preparing download for {selected_format}..."]

        if include_parcels_toggle.value and not parcels_requested_and_clipped:
            messages_to_display.append("Note: Parcels were requested, but no clipped parcel data is available for this download.")
        if include_imagery_toggle.value and not imagery_requested_and_clipped:
            messages_to_display.append("Note: Imagery was requested, but no clipped imagery data is available for this download.")
        if not has_contours:
             messages_to_display.append("Note: No clipped contour data is available for this download.")

        # Prepare GeoDataFrames for export
        export_contours_gdf = None
        if has_contours:
            try: export_contours_gdf = clipped_gdf.copy()
            except Exception as e: messages_to_display.append(f"Error copying contour data: {e}"); has_contours = False

        export_parcels_gdf = None
        if parcels_requested_and_clipped:
            try: export_parcels_gdf = gdf_parcels_clipped.copy()
            except Exception as e: messages_to_display.append(f"Error copying parcel data: {e}"); parcels_requested_and_clipped = False


        try:
            with tempfile.TemporaryDirectory() as tmpdir:
                town_name_part = selected_town_names_str.replace(', ', '_').replace(' ', '_') if selected_town_names_str else "multi_town"

                # Define base filenames
                base_fn_contours = f"clipped_{town_name_part}_contours"
                base_fn_parcels = f"clipped_{town_name_part}_parcels"
                base_fn_imagery = f"clipped_{town_name_part}_imagery"
                base_fn_combined_zip = f"clipped_{town_name_part}_data"

                files_to_zip = [] # List of (source_path, arcname_in_zip)
                single_file_to_download = None
                final_display_name = None

                # --- GeoJSON ---
                if selected_format == 'GeoJSON':
                    if has_contours:
                        path = os.path.join(tmpdir, f"{base_fn_contours}.geojson")
                        export_contours_gdf.to_file(path, driver="GeoJSON")
                        files_to_zip.append((path, f"{base_fn_contours}.geojson"))
                    if parcels_requested_and_clipped:
                        path = os.path.join(tmpdir, f"{base_fn_parcels}.geojson")
                        export_parcels_gdf.to_file(path, driver="GeoJSON")
                        files_to_zip.append((path, f"{base_fn_parcels}.geojson"))
                    if imagery_requested_and_clipped:
                        # Imagery is already a file, just need its path and desired name in zip
                        files_to_zip.append((clipped_imagery_path, f"{base_fn_imagery}.tif"))

                    if len(files_to_zip) > 1:
                        final_display_name = f"{base_fn_combined_zip}_geojson.zip"
                    elif len(files_to_zip) == 1:
                        single_file_to_download = files_to_zip[0][0]
                        final_display_name = files_to_zip[0][1]
                        files_to_zip = [] # Clear so it's not zipped

                # --- Shapefile ---
                elif selected_format == 'Shapefile':
                    shp_export_dir = os.path.join(tmpdir, "shapefile_export")
                    os.makedirs(shp_export_dir, exist_ok=True)
                    if has_contours:
                        # (Add column name truncation and type conversion for contours here if needed)
                        export_contours_gdf.to_file(os.path.join(shp_export_dir, f"{base_fn_contours}.shp"), driver="ESRI Shapefile")
                    if parcels_requested_and_clipped:
                        # (Add column name truncation and type conversion for parcels here if needed)
                        export_parcels_gdf.to_file(os.path.join(shp_export_dir, f"{base_fn_parcels}.shp"), driver="ESRI Shapefile")
                    if imagery_requested_and_clipped:
                        shutil.copy(clipped_imagery_path, os.path.join(shp_export_dir, f"{base_fn_imagery}.tif"))

                    # All shapefile components and imagery are in shp_export_dir, now zip this directory
                    if os.listdir(shp_export_dir): # Check if anything was actually saved
                        zip_path_base = os.path.join(tmpdir, base_fn_combined_zip)
                        shutil.make_archive(zip_path_base, 'zip', shp_export_dir)
                        single_file_to_download = zip_path_base + ".zip"
                        final_display_name = f"{base_fn_combined_zip}_shapefiles.zip"
                    else:
                        messages_to_display.append("No data was available to create Shapefiles.")


                # --- GeoPackage ---
                elif selected_format == 'GeoPackage':
                    gpkg_path = os.path.join(tmpdir, f"{base_fn_combined_zip}.gpkg")
                    gpkg_has_vector = False
                    if has_contours:
                        export_contours_gdf.to_file(gpkg_path, driver="GPKG", layer="clipped_contours")
                        gpkg_has_vector = True
                    if parcels_requested_and_clipped:
                        export_parcels_gdf.to_file(gpkg_path, driver="GPKG", layer="clipped_parcels", mode='a' if gpkg_has_vector else 'w')
                        gpkg_has_vector = True

                    if imagery_requested_and_clipped:
                        if gpkg_has_vector: # Zip GPKG and TIF
                            files_to_zip.append((gpkg_path, f"{base_fn_combined_zip}.gpkg"))
                            files_to_zip.append((clipped_imagery_path, f"{base_fn_imagery}.tif"))
                            final_display_name = f"{base_fn_combined_zip}_gpkg_and_imagery.zip"
                        else: # Only imagery, no vector for GPKG
                            single_file_to_download = clipped_imagery_path # Download TIF directly
                            final_display_name = f"{base_fn_imagery}.tif"
                            messages_to_display.append("Note: GeoPackage selected, but only imagery is available. Downloading GeoTIFF.")
                    elif gpkg_has_vector: # Only vector for GPKG
                        single_file_to_download = gpkg_path
                        final_display_name = f"{base_fn_combined_zip}.gpkg"
                    else:
                        messages_to_display.append("No data available for GeoPackage.")


                # --- DXF ---
                elif selected_format == 'DXF':
                    if has_contours:
                        dxf_path = os.path.join(tmpdir, f"{base_fn_contours}.dxf")
                        # ... (DXF export logic for contours as in previous versions) ...
                        try:
                            doc = ezdxf.new(dxfversion='R2010')
                            msp = doc.modelspace()
                            if not doc.layers.has_entry("CONTOURS"): doc.layers.add(name="CONTOURS", color=7)
                            has_elevation_column = 'Elevation' in export_contours_gdf.columns
                            if not has_elevation_column: messages_to_display.append("DXF Warning: 'Elevation' column not in contours. Z=0 used.")
                            for _, row in export_contours_gdf.iterrows():
                                geom = row.geometry; dxfattribs = {'layer': 'CONTOURS'}
                                if has_elevation_column:
                                    try: dxfattribs['elevation'] = float(row['Elevation'])
                                    except: pass
                                if geom.geom_type == 'LineString':
                                    coords = [(p[0], p[1]) for p in geom.coords] if geom.has_z else list(geom.coords)
                                    if coords: msp.add_lwpolyline(coords, dxfattribs=dxfattribs)
                                elif geom.geom_type == 'MultiLineString':
                                    for line in geom.geoms:
                                        coords = [(p[0], p[1]) for p in line.coords] if line.has_z else list(line.coords)
                                        if coords: msp.add_lwpolyline(coords, dxfattribs=dxfattribs.copy())
                            doc.saveas(dxf_path)
                            single_file_to_download = dxf_path
                            final_display_name = f"{base_fn_contours}.dxf"
                        except Exception as dxf_e: messages_to_display.append(f"DXF Export Error: {dxf_e}")
                    else: messages_to_display.append("DXF: No contour data to export.")
                    if parcels_requested_and_clipped or imagery_requested_and_clipped:
                        messages_to_display.append("Note: Parcels and Imagery are not included in DXF exports.")

                # --- GeoParquet ---
                elif selected_format == 'GeoParquet':
                    if has_contours:
                        path = os.path.join(tmpdir, f"{base_fn_contours}.parquet")
                        export_contours_gdf.to_parquet(path)
                        files_to_zip.append((path, f"{base_fn_contours}.parquet"))
                    if parcels_requested_and_clipped:
                        path = os.path.join(tmpdir, f"{base_fn_parcels}.parquet")
                        export_parcels_gdf.to_parquet(path)
                        files_to_zip.append((path, f"{base_fn_parcels}.parquet"))
                    if imagery_requested_and_clipped:
                         files_to_zip.append((clipped_imagery_path, f"{base_fn_imagery}.tif"))

                    if len(files_to_zip) > 1:
                        final_display_name = f"{base_fn_combined_zip}_geoparquet.zip"
                    elif len(files_to_zip) == 1:
                        single_file_to_download = files_to_zip[0][0]
                        final_display_name = files_to_zip[0][1]
                        files_to_zip = [] # Clear so it's not zipped
                else:
                    messages_to_display.append(f"Unsupported format: {selected_format}")

                # --- Zipping if multiple files ---
                if files_to_zip: # Implies multiple files need to be zipped
                    zip_output_path = os.path.join(tmpdir, final_display_name)
                    with zipfile.ZipFile(zip_output_path, 'w') as zipf:
                        for file_path, arc_name in files_to_zip:
                            zipf.write(file_path, arcname=arc_name)
                    single_file_to_download = zip_output_path # The zip file is the single file to download

                # --- Download ---
                if single_file_to_download and final_display_name:
                    # Copy to a location Colab can access for download, using the display name
                    final_colab_download_path = os.path.join(os.getcwd(), final_display_name)
                    shutil.copy(single_file_to_download, final_colab_download_path)

                    messages_to_display.append(f"File '{final_display_name}' is ready.")
                    with download_output_area:
                        for msg in messages_to_display: print(msg)
                    from google.colab import files
                    files.download(final_colab_download_path)
                elif not files_to_zip and not single_file_to_download : # No files were prepared
                     messages_to_display.append("No data processed for the selected format or an error occurred.")
                     with download_output_area:
                        for msg in messages_to_display: print(msg)


        except Exception as e:
            with download_output_area:
                for msg in messages_to_display: print(msg)
                print(f"An unexpected error occurred during download preparation: {e}")
                import traceback
                traceback.print_exc()
        finally:
            # Clean up the global path to the clipped imagery if it exists,
            # as it was created outside the main tempdir for this download function.
            # The temp_processing_dir in Cell 1 is where it's actually stored.
            # This download function uses its own tmpdir for zipping etc.
            # The actual clipped_imagery_path file will be cleaned up when its parent temp_processing_dir is cleaned.
            pass


    if hasattr(download_button, '_click_handlers') and download_button._click_handlers:
        download_button._click_handlers.callbacks = []

    download_button.on_click(on_download_button_clicked)

    # Need to import zipfile if it's not already
    import zipfile
    display(widgets.VBox([output_format_dropdown, download_button, download_output_area]))
